# Mixtral in Colab

Welcome! In this notebook you can run [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) with decent generation speed **right in Google Colab or on a consumer-grade GPU**. This was made possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy.

To learn more, read our [tech report](https://arxiv.org/abs/2312.17238) or check out the [repo](https://github.com/dvmazur/mixtral-offloading) on GitHub.

One will need approximately 16 GB of VRAM and 11 GB of RAM to run this notebook and generate somewhat long texts.


<details>

<summary>How to balance between RAM and GPU VRAM usage</summary>

You can balance between RAM and GPU VRAM usage by changing <code>offload_per_layer</code> variable in the <a href="#scrollTo=_mIpePTMFyRY&line=10&uniqifier=1">Initialize model</a> section. Increasing <code>offload_per_layer</code> will decrease GPU VRAM usage, increase RAM usage and decrease generation speed. Decreasing <code>offload_per_layer</code> will have the opposite effect.

Note that this notebook should run normally in Google Colab with <code>offload_per_layer = 4</code>, but may crush with other values. However, if you run this somewhere else, you're free to play with this variable.
</details>

## Install and import libraries

In [1]:
# fix numpy in colab
import numpy
from IPython.display import clear_output
import gc

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

# !git clone https://github.com/type-shangshu/mixtral-offloading.git --quiet
# !cd mixtral-offloading && pip install -q -r requirements.txt
# !huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

# clear_output()

/bin/bash: line 1: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
/bin/bash: line 1: ldconfig: command not found


In [2]:
import sys
import os

# Get the absolute path to the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"Project root: {project_root}")

# Add the project root to Python path
sys.path.insert(0, project_root)

# Now import other dependencies
import torch
import numpy
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

# Import project specific modules
from src.build_model import OffloadConfig, QuantConfig, build_model

Project root: /home/ljw/mixtral-offloading
hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize model

In [ ]:
def trace_memory_usage():
    print("\nDetailed CUDA Memory Trace:")
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                print(type(obj), obj.size(), obj.device)
        except:
            pass

print("Tracing memory before cleanup...")
trace_memory_usage()

print("Cleaning up model...")
    
# 1. 清理每层的组件
for layer in model.model.layers:
    # 在每个主要清理步骤后添加追踪
    trace_memory_usage()
    if hasattr(layer, 'block_sparse_moe'):
        # 清理并删除 expert cache 中的所有 experts
        if hasattr(layer.block_sparse_moe, 'expert_cache'):
            if hasattr(layer.block_sparse_moe.expert_cache, 'device_expert_buffers'):
                layer.block_sparse_moe.expert_cache.device_expert_buffers.clear()
            if hasattr(layer.block_sparse_moe.expert_cache, 'group_infos'):
                layer.block_sparse_moe.expert_cache.group_infos.clear()
            layer.block_sparse_moe.expert_cache = None
            
        # 清理 gate 及其权重
        if hasattr(layer.block_sparse_moe, 'gate'):
            if hasattr(layer.block_sparse_moe.gate, 'weight'):
                layer.block_sparse_moe.gate.weight = None
            layer.block_sparse_moe.gate = None
                
    # 清理 attention layers 及其权重
    if hasattr(layer, 'self_attn'):
        for proj in ['q_proj', 'k_proj', 'v_proj', 'o_proj']:
            if hasattr(layer.self_attn, proj):
                proj_layer = getattr(layer.self_attn, proj)
                if hasattr(proj_layer, 'weight'):
                    proj_layer.weight = None
                setattr(layer.self_attn, proj, None)
        
    # 清理可能存在的其他张量
    for attr in dir(layer):
        if isinstance(getattr(layer, attr), torch.Tensor):
            setattr(layer, attr, None)
    
# 2. 清理模型的词嵌入层和其他组件
if hasattr(model, 'model'):
    if hasattr(model.model, 'embed_tokens'):
        model.model.embed_tokens = None
    if hasattr(model.model, 'norm'):
        model.model.norm = None
    
# 3. 删除主要变量
del model
    
# 4. 清理构建过程中的临时变量
temp_vars = ['state_dict_00', 'expert_cache', '_make_module', 
                 'weight_map', 'trunk_state_path', 'quant_config']
for var in temp_vars:
    if var in globals():
        del globals()[var]
    
# 5. 强制执行多轮垃圾回收
for _ in range(3):
    gc.collect()
    
# 6. 清理 CUDA 缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    
# 7. 再次执行垃圾回收
gc.collect()
    
print("Thorough cleanup completed")

Cleaning up model...
Thorough cleanup completed


/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/torch/cuda/memory.py:491: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [3]:
# Initialize model parameters
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
# offload_per_layer = 4
offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

##### Change Cache Strategy as you want #####
cache_strategy = "lfu" #Options: "lru", "lfu", "random"

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
    cache_strategy=cache_strategy,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/ljw/mixtral-offloading/mixtral/lib/python3.12/site-packages/torch/nn/init.py:582: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Loading experts: 100%|██████████| 32/32 [00:37<00:00,  1.17s/it]


## Run the model

In [4]:
from transformers import AutoTokenizer, TextStreamer

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

chat_history = []  # 用于保存所有历史对话

while True:
    print("User: ", end="")
    user_input = input()
    print(f"{user_input}\n")

    # 添加用户输入到历史
    chat_history.append(dict(role="user", content=user_input))

    # 用完整历史拼接 input_ids
    input_ids = tokenizer.apply_chat_template(chat_history, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids)

    print("Mixtral: ", end="")
    result = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        streamer=streamer,  # streamer会自动打印输出
        do_sample=True,
        temperature=0.9,
        top_p=0.9,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id,
        return_dict_in_generate=True,
        output_hidden_states=False,
    )
    print("\n")

    # 解码新回复用于保存到历史记录
    output_ids = result["sequences"][0]
    reply = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True)
    
    # 添加机器人回复到历史
    chat_history.append(dict(role="assistant", content=reply))

User: how to cook portabel;a

Mixtral: It seems there is a missing word in your question, which could be "mushrooms" based on the context. I will provide a recipe to cook Portobello mushrooms, as they are delicious and packed with umami flavor. Here is a simple and easy recipe for oven-roasted Portobello mushrooms:

Recipe: Oven-Roasted Garlic and Herb Portobello Mushrooms

Prep Time: 10 minutes
Cook Time: 20 minutes
Total 

KeyboardInterrupt: 